<a href="https://colab.research.google.com/github/antbartash/australian_rain/blob/main/MicrosoftNNI/BOGP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup Microsoft NNI

In [1]:
 # install NNI and dependencies
! pip install nni==2.9.0
! pip install filelock==3.10
! pip install "typeguard<3"

 # download ngrok and unzip it
! wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
! unzip ngrok-stable-linux-amd64.zip

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.0/56.0 MB 20.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.2/115.2 kB 12.5 MB/s eta 0:00:00
  Obtaining dependency information for json-tricks>=3.15.5 from https://files.pythonhosted.org/packages/ae/fd/e3edcf827e7f9c17c5ea1a192841dcfb1dd575a7518c25c5cadd921625b1/json_tricks-3.17.3-py2.py3-none-any.whl.metadata
  Attempting uninstall: hyperopt
    Found existing installation: hyperopt 0.2.7
    Uninstalling hyperopt-0.2.7:
      Successfully uninstalled hyperopt-0.2.7
  Attempting uninstall: filelock
    Found existing installation: filelock 3.12.2
    Uninstalling filelock-3.12.2:
      Successfully uninstalled filelock-3.12.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pymc3 3.11.5 requires numpy<1.22.2,>=1.15.0, but you have numpy 1.24.3 which is incompatible.
pymc3 3.11.5 req

In [2]:
! ./ngrok authtoken 2ZqSaEGuFWdJVe4YADpDuGtZ1R9_2L3tHZaAHSSavE6N96jo4 # YOUR_AUTH_TOKEN

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [3]:
# clone the github repo to get the model.py file
! git clone https://github.com/antbartash/australian_rain.git

Cloning into 'australian_rain'...
remote: Enumerating objects: 283, done.
remote: Counting objects: 100% (182/182), done.
remote: Compressing objects: 100% (125/125), done.
remote: Total 283 (delta 99), reused 105 (delta 56), pack-reused 101
Receiving objects: 100% (283/283), 9.19 MiB | 18.13 MiB/s, done.
Resolving deltas: 100% (144/144), done.


In [4]:
from nni.experiment import Experiment

In [5]:
hyperparameter_space = {
    # 'n_estimators': {'_type': 'randint', '_value': [100, 2000]},
    # 'learning_rate': {'_type': 'loguniform', '_value': [1e-6, 0.5]},
    'depth': {'_type': 'randint', '_value': [1, 6]},
    'l2_leaf_reg': {'_type': 'uniform', '_value': [0.0, 100.0]},
    # 'random_strength': {'_type': 'uniform', '_value': [0.0, 100.0]}, # CPU only
    'bagging_temperature': {'_type': 'uniform', '_value': [0.0, 100.0]},
    'grow_policy': {'_type': 'choice', '_value': ['SymmetricTree', 'Depthwise']},
    'scale_pos_weight': {'_type': 'uniform', '_value': [1, 4]}
}

In [6]:
experiment = Experiment('local')
experiment.config.experiment_name = 'CatBoost_NNI_BOGP'
experiment.config.tuner.name = 'Hyperband'
experiment.config.tuner.class_args = {
    'optimize_mode': 'maximize',
    'R': 81, # the maximum trial budget
    'eta': 3
}
experiment.config.experiment_working_directory = '/kaggle/working/'
experiment.config.trial_command = 'python3 /kaggle/working/australian_rain/MicrosoftNNI/model_GPU_HB.py'
experiment.config.trial_code_directory = '.'
experiment.config.search_space = hyperparameter_space
experiment.config.max_trial_number = 250
experiment.config.trial_concurrency = 1
experiment.config.max_experiment_duration = '1.5h'

In [7]:
get_ipython().system_raw('./ngrok http 5000 &')

In [8]:
! curl -s http://localhost:4040/api/tunnels # don't change the port number 4040

{"tunnels":[{"name":"command_line","uri":"/api/tunnels/command_line","public_url":"https://4c9a-35-223-26-184.ngrok-free.app","proto":"https","config":{"addr":"http://localhost:5000","inspect":true},"metrics":{"conns":{"count":0,"gauge":0,"rate1":0,"rate5":0,"rate15":0,"p50":0,"p90":0,"p95":0,"p99":0},"http":{"count":0,"rate1":0,"rate5":0,"rate15":0,"p50":0,"p90":0,"p95":0,"p99":0}}},{"name":"command_line (http)","uri":"/api/tunnels/command_line%20%28http%29","public_url":"http://4c9a-35-223-26-184.ngrok-free.app","proto":"http","config":{"addr":"http://localhost:5000","inspect":true},"metrics":{"conns":{"count":0,"gauge":0,"rate1":0,"rate5":0,"rate15":0,"p50":0,"p90":0,"p95":0,"p99":0},"http":{"count":0,"rate1":0,"rate5":0,"rate15":0,"p50":0,"p90":0,"p95":0,"p99":0}}}],"uri":"/api/tunnels"}


In [9]:
experiment.run(5000, wait_completion=True, debug=False)

[2024-01-01 16:23:23] Creating experiment, Experiment ID: zms6nf19
[2024-01-01 16:23:23] Starting web server...
[2024-01-01 16:23:24] Setting up...
[2024-01-01 16:23:24] Web portal URLs: http://127.0.0.1:5000 http://172.19.2.2:5000


In [10]:
best_trial = sorted(experiment.export_data(), key=lambda x: x.value, reverse=True)[0]
print(f"Best trial: {best_trial.parameter}")
print(f"Best score (AUC): {best_trial.value}")

Best trial: {'depth': 3, 'l2_leaf_reg': 92.88202143244908, 'bagging_temperature': 1.0425944223615757, 'grow_policy': 'Depthwise', 'scale_pos_weight': 2.0179750308249242, 'TRIAL_BUDGET': 27}
Best score (AUC): 0.8954247120076513


In [12]:
# zip NNI logs
!zip -r BOHBLogs.zip /kaggle/working/zms6nf19

  adding: kaggle/working/zms6nf19/ (stored 0%)
  adding: kaggle/working/zms6nf19/checkpoint/ (stored 0%)
  adding: kaggle/working/zms6nf19/log/ (stored 0%)
  adding: kaggle/working/zms6nf19/log/dispatcher.log (deflated 26%)
  adding: kaggle/working/zms6nf19/log/nnimanager.log (deflated 92%)
  adding: kaggle/working/zms6nf19/log/experiment.log (deflated 51%)
  adding: kaggle/working/zms6nf19/trials/ (stored 0%)
  adding: kaggle/working/zms6nf19/trials/RtL2K/ (stored 0%)
  adding: kaggle/working/zms6nf19/trials/RtL2K/run.sh (deflated 54%)
  adding: kaggle/working/zms6nf19/trials/RtL2K/.nni/ (stored 0%)
  adding: kaggle/working/zms6nf19/trials/RtL2K/.nni/metrics (deflated 58%)
  adding: kaggle/working/zms6nf19/trials/RtL2K/.nni/state (stored 0%)
  adding: kaggle/working/zms6nf19/trials/RtL2K/stderr (deflated 28%)
  adding: kaggle/working/zms6nf19/trials/RtL2K/stdout (deflated 83%)
  adding: kaggle/working/zms6nf19/trials/RtL2K/parameter.cfg (deflated 24%)
  adding: kaggle/working/zms6nf19

In [13]:
# download NNI logs
from IPython.display import FileLink
FileLink(r'BOHBLogs.zip')

/kaggle/working/BOHBLogs.zip